In [1]:
import numpy as np

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over = 'ignore') #Ignore warnings about overflow data points

Loading BokehJS ...

In [39]:
# Set up constant in SI units
dens = 8.47e+28
rho = 1.56e-8
omega0 = 1/rho
m = 9.109e-31
elec = 1.602e-19
tau = m / (dens*rho*elec**2)
freq_min = 1e-12   #Lower limit of frequency (approximately DC scenario)
freq_max = 5000.0    #Upper limit of frequency
N = 5000   #Number of points for the independent variable

# Set up the variable space
freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
w = 2*np.pi*freq*1e+12 #Convert frequency to angular frequency
omega1 = omega0 / (1.0 + w**2*tau**2)  #Real part of the conductivity
omega2 = w*tau*omega0 / (1.0 + w**2*tau**2)  #imaginary part of the conductivity

In [40]:
#calculate permittivity
freq2 = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
w = 2*np.pi*freq*1e+12 #Convert frequency to angular frequency
eps = 8.854e-12 #Vacuum permittivity
eps0 = 1.0 #Effective permittivity of the material
eps1 = eps0 - omega2 /(eps*w)
eps2 = omega1/(eps*w)

In [58]:
# Set up conductivity plot
plot = figure(plot_height=200, plot_width=400, title="Real part of AC conductivity",
              tools="pan,reset,save,wheel_zoom",
              y_axis_type="log", x_range=[freq_min, freq_max], y_range=[1e+4, 8e+7])
plot.xaxis[0].axis_label='Frequency (THz)'
plot.yaxis[0].axis_label='conductivity (Ohm-1m-1)'
r = plot.line(freq, omega1, line_width=3, line_alpha=0.6)

plot2 = figure(plot_height=200, plot_width=400, title="Imaginary part of AC conductivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max], y_range=[0.0, 2e+7])
plot2.xaxis[0].axis_label='Frequency (THz)'
plot2.yaxis[0].axis_label='conductivity (Ohm-1m-1)'
rr = plot2.line(freq, omega2, line_width=3, line_alpha=0.6, color = 'red')

In [62]:
# Set up permittivity plot
plot3 = figure(plot_height=200, plot_width=400, title="Real part of permittivity",
              tools="pan,reset,save,wheel_zoom",
              x_axis_type="log", x_range=[freq_min, freq_max], y_range=[-6e+6, 5e+4])
plot3.xaxis[0].axis_label='Frequency (THz)'
plot3.yaxis[0].axis_label='Permittivity (F/m)'
rrr = plot3.line(freq, eps1, line_width=3, line_alpha=0.6)

plot4 = figure(plot_height=200, plot_width=400, title= "imaginary part of permittivity",
              tools="pan,reset,save,wheel_zoom",
              x_axis_type="log", x_range=[freq_min, freq_max], y_range=[0, 6.5e+20])
plot4.xaxis[0].axis_label='Frequency (THz)'
plot4.yaxis[0].axis_label='Permittivity (F/m)'
rrrr = plot4.line(freq, eps2, line_width=3, line_alpha=0.6, color = 'red')

In [63]:
# Set up callbacks to update the plots
def update_data(density, resistivity):
# Generate the new curve
    dens = density*1e+28
    rho = resistivity*1e-8
    omega0 = 1/rho
    tau = m / (dens*rho*elec**2)
    freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
    w = 2*np.pi*freq*1e+9 #Convert frequency to angular frequency
    r.data_source.data['y'] = omega0 / (1.0 + w**2*tau**2) 
    rr.data_source.data['y'] = w*tau*omega0 / (1.0 + w**2*tau**2)
    rrr.data_source.data['y'] = eps0 - rr.data_source.data['y']/(eps*w)
    rrrr.data_source.data['y'] =  r.data_source.data['y']/(eps*w)

    push_notebook()

In [64]:
show(column([row(plot,plot2),row(plot3, plot4)]),notebook_handle=True)

In [65]:
interact(update_data, density = (0.091, 24.7, 0.02), resistivity = (2.8, 107, 0.5))

interactive(children=(FloatSlider(value=12.391, description='density', max=24.7, min=0.091, step=0.02), FloatS…

<function __main__.update_data(density, resistivity)>

In [18]:
eps2

array([1.15227467e+18, 1.11976862e+06, 5.16892982e+05, ...,
       5.07144034e-03, 5.06382942e-03, 5.05623372e-03])